# Ajustar (tunear) Hiperparámetros

Texto extraído de `ChatGPT`:

El **tuneo de hiperparámetros** (o **ajuste de hiperparámetros**) es el proceso de seleccionar los valores óptimos para los **hiperparámetros** de un modelo de aprendizaje automático (machine learning). Los hiperparámetros son parámetros que **no se aprenden** durante el proceso de entrenamiento, sino que deben ser definidos **antes** de comenzar el entrenamiento del modelo.

## Ejemplos de hiperparámetros comunes:
- **Tasa de aprendizaje**: Controla qué tan grande es el paso que da el modelo en cada iteración durante el entrenamiento.
- **Número de capas y neuronas** (en redes neuronales): Define la estructura de la red, como cuántas capas ocultas y cuántas neuronas tiene cada capa.
- **Tamaño del lote (batch size)**: Número de ejemplos de entrenamiento que se procesan antes de actualizar los pesos del modelo.
- **Número de árboles** (en Random Forest): Especifica cuántos árboles se entrenarán en un modelo de Random Forest.
- **C** (en SVM): Parámetro de regularización que controla el margen entre las clases y el costo de cometer errores en un modelo de Support Vector Machine.

## Métodos para hacer tuneo de hiperparámetros:
1. **Búsqueda en cuadrícula (Grid Search)**: Es un método exhaustivo que prueba todas las combinaciones posibles de hiperparámetros dentro de un rango definido. Es muy preciso pero puede ser muy costoso computacionalmente.
   
2. **Búsqueda aleatoria (Random Search)**: En lugar de probar todas las combinaciones posibles, selecciona aleatoriamente un subconjunto de combinaciones de hiperparámetros. Aunque no garantiza encontrar la mejor combinación, suele ser más eficiente que la búsqueda en cuadrícula, especialmente cuando hay muchos hiperparámetros.

3. **Optimización bayesiana**: Utiliza un enfoque probabilístico para seleccionar las combinaciones de hiperparámetros que es más probable que conduzcan a un buen rendimiento, basándose en resultados anteriores.

4. **Algoritmos genéticos**: Utilizan principios de la evolución biológica, como la selección natural y el cruce, para explorar de manera eficiente el espacio de hiperparámetros.

## Objetivo del tuneo de hiperparámetros:
El objetivo principal del tuneo de hiperparámetros es encontrar la configuración que maximice el rendimiento del modelo, es decir, obtener el modelo que **generalice mejor** sobre datos no vistos. Esto se suele hacer evaluando el rendimiento del modelo con cada conjunto de hiperparámetros en un conjunto de **validación** y seleccionando aquellos que dan mejores resultados.

## Conclusión:
El tuneo de hiperparámetros es un paso crucial para obtener un modelo de **machine learning** de alto rendimiento, ya que los valores de los hiperparámetros pueden tener un gran impacto en la capacidad del modelo para aprender y generalizar correctamente.


## Ejemplo en Python

### Contexto y Datos

In [1]:
import pandas as pd

In [4]:
personas = pd.read_csv("../datos/ingresos.csv")
personas.head() # ingreso -> 0: ingreso bajo, 1: ingreso alto

,edad,estudio,genero,tipo_trabajo,horas,ingreso
0,27,9,0,0,40,0
1,30,9,0,0,40,0
2,34,8,0,0,40,0
3,26,13,0,0,50,0
4,23,4,1,0,25,0


In [5]:
personas.shape

(100, 6)

### Ejemplo con Bosques Aleatorios

Como fue mencionado más arriba en la sección explicativa/teórica, uno de los típicos casos en los que se debe experimentar y probar distintos parámetros hasta encontrar aquellos que nos brinden un modelo eficiente, es el de los bosques aleatorios.

Entre otros parámetros que se deben revisar y ajustar entre cada prueba se encuentra la cantidad de árboles que integre el bosque, la cantidad de características (columnas) que integrarán los nodos de cada árbol, el criterio que se tomará como métrica a la hora de decidir generar un nodo en particular a partir de una columnas, etc.

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
bosque = RandomForestClassifier()
bosque.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

#### Búsqueda de cuadrícula (o Rejilla)

In [8]:
from sklearn.model_selection import GridSearchCV

Armamos un diccionario para explorar las distintas posibilidades que pueden tomar los parámetros que más nos interesa "tunear". Por ejemplo, podemos querer probar la eficiencia de un bosque con un criterio de creación de nodos usando la impureza de Gini, y otros casos con la entropía. Casos de bosques con 10 árboles, otro con 20 y otro con 30. Y también el número de observaciones que se tomará para el entrenamiento puede variar en cada prueba.

Por supuesto, tener en cuenta que la cantidad y diversidad de valores para probar y ajustar los hiperparámetros en este caso son solo de ejemplo.

In [16]:
parametros = {
    "criterion": ("gini", "entropy"), # 2 posibilidades
    "n_estimators": (10, 20, 30), # 3 posibilidades
    "max_samples": (1/3, 2/3) # 2 posibilidades
} 
# tenemos 2 * 3 * 2 = 12 posiblidades diferentes para probar de combinaciones en este caso

rejilla = GridSearchCV(
    bosque,
    parametros,
    scoring="accuracy" # metrica que nos interesa medir en cuanto a eficiencia
)

rejilla.fit(personas[personas.columns[:-1]].values, 
            personas["ingreso"].values)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_samples': (0.3333333333333333,
                                         0.6666666666666666),
                         'n_estimators': (10, 20, 30)},
             scoring='accuracy')

#### Información de la Búsqueda en Rejilla

In [17]:
sorted(rejilla.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_criterion',
 'param_max_samples',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

Como vemos, la rejilla nos devuelve como resultado un conjunto de métricas en función de las pruebas realizadas con los diferentes valores de los hiperparámetros. 

Aquellos que dicen `param_`... hacen referencia a los hiperparámetros que definimos en el diccionario de más arriba.

Por otro lado, algo importante a tener en cuenta es que la rejilla utiliza la validación cruzada, por defecto con 5 pliegues.
Los que dicen `split(nro)_test_score` hacen referencia a las métricas de cada combinación para cada caso de validación cruzada.

#### Rejilla de Parámetros explorados

In [18]:
rejilla.cv_results_["params"] # nos muestra las combinaciones probadas y exploradas

[{'criterion': 'gini', 'max_samples': 0.3333333333333333, 'n_estimators': 10},
 {'criterion': 'gini', 'max_samples': 0.3333333333333333, 'n_estimators': 20},
 {'criterion': 'gini', 'max_samples': 0.3333333333333333, 'n_estimators': 30},
 {'criterion': 'gini', 'max_samples': 0.6666666666666666, 'n_estimators': 10},
 {'criterion': 'gini', 'max_samples': 0.6666666666666666, 'n_estimators': 20},
 {'criterion': 'gini', 'max_samples': 0.6666666666666666, 'n_estimators': 30},
 {'criterion': 'entropy',
  'max_samples': 0.3333333333333333,
  'n_estimators': 10},
 {'criterion': 'entropy',
  'max_samples': 0.3333333333333333,
  'n_estimators': 20},
 {'criterion': 'entropy',
  'max_samples': 0.3333333333333333,
  'n_estimators': 30},
 {'criterion': 'entropy',
  'max_samples': 0.6666666666666666,
  'n_estimators': 10},
 {'criterion': 'entropy',
  'max_samples': 0.6666666666666666,
  'n_estimators': 20},
 {'criterion': 'entropy',
  'max_samples': 0.6666666666666666,
  'n_estimators': 30}]

#### Resultados de la Búsqueda en Rejilla

In [19]:
print(rejilla.cv_results_["rank_test_score"])
print(rejilla.cv_results_["mean_test_score"])
print(rejilla.best_score_)
print(rejilla.best_params_)

[ 2  4  4  6  9 10  8  6  1 10  3 10]
[0.82 0.8  0.8  0.79 0.77 0.76 0.78 0.79 0.82 0.76 0.81 0.76]
0.8200000000000001
{'criterion': 'entropy', 'max_samples': 0.3333333333333333, 'n_estimators': 30}


#### Usando/extrayendo el mejor modelo

Por defecto, si llamamos al método `predict` directamente usando nuestra instancia de `rejilla`, llamará al modelo con los parámetros y valores que arrojaron las mejores métricas. Sin embargo, si quisiéramos obtener el bosque correspondiente a ese mejor modelo para trabajar con él por separado, es posible hacerlo:

In [21]:
print(rejilla.predict([[50, 16, 1, 1, 40]]))

mejor_bosque = rejilla.best_estimator_
print(mejor_bosque.predict([[50, 16, 1, 1, 40]]))

[1]
[1]


#### Búsqueda Aleatoria de Hiperparámetros

Como se ha mencionado en la explicación teórica de más arriba, para aquellos casos en los que se deban explorar muchos hiperparámetros con muchos posibles valores para cada uno, o bien tengamos muchos datos, la técnica de la búsqueda de rejilla podría tornarse muy costosa en tiempos y computacionalmente.

Alternativamente, por este motivo se puede utilizar la búsqueda aleatoria, la cual consiste en explorar SOLO una parte del total de combinaciones de valores de parámetros que le pasemos mediante lo que indiquemos en `n_iter`:

In [22]:
from sklearn.model_selection import RandomizedSearchCV

In [24]:
parametros = {
    "criterion": ("gini", "entropy"), # 2 posibilidades
    "n_estimators": (10, 20, 30), # 3 posibilidades
    "max_samples": (1/3, 2/3) # 2 posibilidades
} 

rejilla_aleatoria = RandomizedSearchCV(
    bosque,
    parametros,
    scoring="accuracy",
    cv=5,
    n_iter=3 # indico que explore solo 3 de las 12 combinaciones
)

rejilla_aleatoria.fit(personas[personas.columns[:-1]].values, 
            personas["ingreso"].values)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=3,
                   param_distributions={'criterion': ('gini', 'entropy'),
                                        'max_samples': (0.3333333333333333,
                                                        0.6666666666666666),
                                        'n_estimators': (10, 20, 30)},
                   scoring='accuracy')

In [25]:
print(rejilla_aleatoria.cv_results_["rank_test_score"])
print(rejilla_aleatoria.cv_results_["mean_test_score"])
print(rejilla_aleatoria.best_score_)
print(rejilla_aleatoria.best_params_)

[1 2 3]
[0.83 0.8  0.79]
0.8299999999999998
{'n_estimators': 30, 'max_samples': 0.3333333333333333, 'criterion': 'gini'}


In [27]:
print(rejilla_aleatoria.cv_results_["params"]) # combinaciones aleatorias elegidas

[{'n_estimators': 30, 'max_samples': 0.3333333333333333, 'criterion': 'gini'}, {'n_estimators': 30, 'max_samples': 0.3333333333333333, 'criterion': 'entropy'}, {'n_estimators': 20, 'max_samples': 0.6666666666666666, 'criterion': 'gini'}]
